In [1]:
import re
import pprint
from collections import Counter
import math
import itertools
import json

In [2]:
pp = pprint.PrettyPrinter(indent=2)

In [3]:
def dump_json(filename, json_object):
    with open(filename, "w") as fp:
        json.dump(json_object, fp)

def dump_normas(filename, normas):
    for norma in normas:
        norma['AssuntoGeral'] = list(norma['AssuntoGeral'])
        norma['Tema'] = list(norma['Tema'])
        norma['Indexacao'] = list(norma['Indexacao'])
    dump_json(filename, normas)

In [4]:
def le_arquivo_brs(filename, debug=False):
    DOCUMENT_BOUNDARY = "*** BRS DOCUMENT BOUNDARY ***"
    FIELD_PATTERN =  re.compile("^\.\.(\w+)\:$")

    def split_string(string, split_regex=r'[.]', replace_newline=True):
        return [s.strip().upper() for s in re.split(split_regex, string.replace('\n',' ') if replace_newline else string) if s and s.strip()]

    def remove_newline(norma, fields):
        for field in fields:
            norma[field] = norma[field].replace('\n','') if field in norma else None
        return norma

    def formata_norma(norma):
        norma['AssuntoGeral'] = set(split_string(norma['AssuntoGeral']) if 'AssuntoGeral' in norma else [])
        norma['Indexacao'] = set(split_string(norma['Indexacao'], r'[,.]') if 'Indexacao' in norma else [])
        norma['Tema'] = set(split_string(norma['Tema'], '\n', False) if 'Tema' in norma else [])
        norma = remove_newline(norma, ['Horario', 'DataNorma', 'Ano', 'Norma', 'Numero', 'Origem', 'Relevancia', 'Tipo', 'Atualizacao'])
        return norma

    normas = []
    norma = {}
    with open(filename, encoding='ANSI') as f:
        for line in f:
            m = FIELD_PATTERN.match(line)
            if m:
                current_field = m.group(1)
                if current_field not in norma:
                    norma[current_field] = ""
            elif DOCUMENT_BOUNDARY in line:
                normas.append(formata_norma(norma))
                norma = {}
                if len(normas) % 5000 == 0 and debug:
                    print(f'Encontradas {len(normas)} normas')
            else:
                norma[current_field] += line
    if debug:
        print(f'Encontradas {len(normas)-1} normas')                
    return normas[1:];

In [5]:
filename = "normas.raw"
normas = le_arquivo_brs("normas.raw", True)

Encontradas 5000 normas
Encontradas 10000 normas
Encontradas 15000 normas
Encontradas 20000 normas
Encontradas 25000 normas
Encontradas 28308 normas


### Imprime uma norma de exemplo

In [6]:
pp.pprint(normas[55])

{ 'ADIN': '',
  'Ano': '2020',
  'AssuntoGeral': { 'CALAMIDADE PÚBLICA',
                    'CRIANÇA E ADOLESCENTE',
                    'IDOSO',
                    'MULHER',
                    'PESSOA COM DEFICIÊNCIA',
                    'SAÚDE PÚBLICA',
                    'SEGURANÇA PÚBLICA'},
  'Atualizacao': '20200622',
  'Complemento': '',
  'DataNorma': '20200522',
  'Ementa': 'Dispõe sobre o registro de ocorrência e o pedido de medida '
            'protetiva de\n'
            'urgência relativos a ato de violência doméstica e familiar contra '
            'a\n'
            'mulher durante o estado de calamidade pública em decorrência da\n'
            'pandemia de Covid-19, causada pelo coronavírus, e dá outras\n'
            'providências.\n',
  'Evento': '',
  'Fonte': '<a '
           'href="http://jornal.iof.mg.gov.br/xmlui/handle/123456789/234547">Publicação</a> '
           '- Minas Gerais Diário do Executivo - 23/05/2020 Pág. 1 Col. 2\n'
           '\n',
  'Horario'

### Funções utilitárias para impressão de atributos

In [7]:
def print_stats_multiplo(normas, atributo='AssuntoGeral', top=20):
    atributoCount = Counter([atr for norma in normas for atr in norma[atributo]])
    paresAtributoCount = Counter([parAtr for norma in normas for parAtr in itertools.combinations(sorted(norma[atributo]), 2)])
    print(f"Quantidade de normas {len(normas)}")
    print(f"Quantidade de normas com {atributo}: {len([norma for norma in normas if len(norma[atributo]) > 0])}")
    print(f"Quantidade de {atributo}: {len(atributoCount)}")
    print(f"Quantidade de {atributo} em mais de 5 normas: {len({x: c for x, c in atributoCount.items() if c >= 5})}")
    print(f"Quantidade de normas com mais de 1 {atributo}: {len([norma for norma in normas if len(norma[atributo]) > 1])}")
    print(f"{atributo} comuns:")
    pp.pprint(atributoCount.most_common(top))
    print(f"\nPares de {atributo} comuns:")
    pp.pprint(paresAtributoCount.most_common(top))

def print_stats_individual(normas, atributo='Ano', top=20):
    atributoCount = Counter([norma[atributo] for norma in normas])
    print(f"Quantidade de normas {len(normas)}")
    print(f"Quantidade de {atributo}: {len(atributoCount)}")
    print(f"Quantidade de {atributo} em mais de 5 normas: {len({x: c for x, c in atributoCount.items() if c >= 5})}")
    pp.pprint(atributoCount.most_common(top))

In [8]:
print_stats_multiplo(normas)

Quantidade de normas 28308
Quantidade de normas com AssuntoGeral: 24560
Quantidade de AssuntoGeral: 629
Quantidade de AssuntoGeral em mais de 5 normas: 192
Quantidade de normas com mais de 1 AssuntoGeral: 3820
AssuntoGeral comuns:
[ ('UTILIDADE PÚBLICA', 12217),
  ('PRÓPRIO PÚBLICO', 2184),
  ('IMÓVEL', 2039),
  ('ESTABELECIMENTO DE ENSINO', 1900),
  ('CRÉDITO', 1485),
  ('TRIBUTOS', 558),
  ('SECRETARIA DE ESTADO DE EDUCAÇÃO (SEE), PESSOAL', 499),
  ('EXECUTIVO, PESSOAL', 368),
  ('DIVISÃO ADMINISTRATIVA', 346),
  ('SAÚDE PÚBLICA', 324),
  ('HOMENAGEM', 240),
  ('MEIO AMBIENTE', 224),
  ('TRANSPORTE E TRÂNSITO', 199),
  ('CALENDÁRIO', 193),
  ('AUXÍLIO FINANCEIRO', 190),
  ('FINANÇAS PÚBLICAS', 189),
  ('CONSTITUIÇÃO ESTADUAL', 173),
  ('EDUCAÇÃO', 158),
  ('AGROPECUÁRIA', 157),
  ('ADMINISTRAÇÃO ESTADUAL, ORÇAMENTO', 154)]

Pares de AssuntoGeral comuns:
[ (('ESTABELECIMENTO DE ENSINO', 'PRÓPRIO PÚBLICO'), 1368),
  ( ( 'ESTABELECIMENTO DE ENSINO',
      'SECRETARIA DE ESTADO DE EDUCAÇ

In [9]:
print_stats_individual(normas)

Quantidade de normas 28308
Quantidade de Ano: 173
Quantidade de Ano em mais de 5 normas: 167
[ ('2007', 760),
  ('2010', 733),
  ('1965', 726),
  ('2009', 700),
  ('2008', 687),
  ('2006', 684),
  ('2011', 600),
  ('2012', 598),
  ('2005', 544),
  ('2014', 540),
  ('2013', 529),
  ('2016', 522),
  ('2004', 500),
  ('2003', 461),
  ('1968', 458),
  ('1992', 430),
  ('2002', 426),
  ('1996', 414),
  ('2018', 409),
  ('2000', 398)]


### Pré-processa assuntos em sub-assuntos e agrupa assuntos comuns

Padroniza nome de assuntos comuns: TRIBUTOS e TRIBUTO para TRIBUTOS

Por exemplo, quebra EXECUTIVO, PESSOAL em EXECUTIVO e PESSOAL

In [10]:
def quebra_em_set(lista, char=','):
    return set([t.strip() for tokens in lista for t in tokens.split(char)])

def substitui_em_set(lista, tokens_antigos, token_novo):
    return set([token_novo if token in tokens_antigos else token for token in lista])

Substitui termos comuns parecidos removendo a vírgula:

In [11]:
for norma in normas:
    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'INDÚSTRIA COMÉRCIO E SERVIÇOS', 
        'INDÚSTRIA, COMÉRCIO E SERVIÇOS',
        'INDÚSTRIA, COMÉRCIO E SERVIÇO'],
        'INDÚSTRIA COMÉRCIO E SERVIÇO'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'TRABALHO, EMPREGO E RENDA'],
        'TRABALHO EMPREGO E RENDA'
    )

In [12]:
for norma in normas:
    norma['AssuntoGeral'] = quebra_em_set(norma['AssuntoGeral'])

In [13]:
print_stats_multiplo(normas)

Quantidade de normas 28308
Quantidade de normas com AssuntoGeral: 24560
Quantidade de AssuntoGeral: 330
Quantidade de AssuntoGeral em mais de 5 normas: 186
Quantidade de normas com mais de 1 AssuntoGeral: 5208
AssuntoGeral comuns:
[ ('UTILIDADE PÚBLICA', 12220),
  ('PRÓPRIO PÚBLICO', 2191),
  ('IMÓVEL', 2040),
  ('ESTABELECIMENTO DE ENSINO', 1906),
  ('PESSOAL', 1797),
  ('CRÉDITO', 1485),
  ('TRIBUTOS', 558),
  ('EXECUTIVO', 553),
  ('ORGANIZAÇÃO ADMINISTRATIVA', 545),
  ('SECRETARIA DE ESTADO DE EDUCAÇÃO (SEE)', 537),
  ('ADMINISTRAÇÃO ESTADUAL', 349),
  ('DIVISÃO ADMINISTRATIVA', 346),
  ('SAÚDE PÚBLICA', 324),
  ('HOMENAGEM', 241),
  ('MEIO AMBIENTE', 224),
  ('JUDICIÁRIO', 216),
  ('ORÇAMENTO', 204),
  ('TRANSPORTE E TRÂNSITO', 199),
  ('CALENDÁRIO', 193),
  ('AUXÍLIO FINANCEIRO', 190)]

Pares de AssuntoGeral comuns:
[ (('ESTABELECIMENTO DE ENSINO', 'PRÓPRIO PÚBLICO'), 1368),
  (('PESSOAL', 'SECRETARIA DE ESTADO DE EDUCAÇÃO (SEE)'), 517),
  (('ESTABELECIMENTO DE ENSINO', 'PESSOAL'

In [14]:
# Agrupa assuntos semelhantes em um único:
for norma in normas:
    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'PROPRIO PÚBLICO'], 
        'PRÓPRIO PÚBLICO'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'ÚTILIDADE PÚBLICA'],
        'UTILIDADE PÚBLICA'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'ADMINISTRAÇÃO PÚBLICA ESTADUAL'],
        'ADMINISTRAÇÃO ESTADUAL'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'ADMINISTRAÇÃO DE ESTÁDIOS DO ESTADO DE MINAS GERAIS'], 
        'ADMINISTRAÇÃO DE ESTÁDIOS DO ESTADO DE MINAS GERAIS (ADEMG)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'ASSEMBLEIA LEGISLATIVA DO ESTADO DE MINAS GERAIS'],
        'ASSEMBLEIA LEGISLATIVA DO ESTADO DE MINAS GERAIS (ALMG)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'AUDITORIA-GERAL DO ESTADO'],
        'AUDITORIA-GERAL DO ESTADO (AUGE)',
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'BENEFICIO PESSOAL'],
        'BENEFÍCIO PESSOAL',
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'CIÊNCIA TECNOLOGIA'],
        'CIÊNCIA E TECNOLOGIA'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'CREDITO'],
        'CRÉDITO'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'COMPANHIA DE PROCESSAMENTO DE DADOS DO ESTADO DE MINAS GERAIS (PRODEMGE)'],
        'COMPANHIA DE TECNOLOGIA DA INFORMAÇÃO DO ESTADO DE MINAS GERAIS (PRODEMGE)',
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'CORPO DE BOMBEIROS'],
        'CORPO DE BOMBEIROS MILITAR DO ESTADO DE MINAS GERAIS (CBMMG)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'DEPARTAMENTO ESTADUAL DE OBRAS PÚBLICAS (DEOP-MG)'],
        'DEPARTAMENTO DE OBRAS PÚBLICAS DO ESTADO DE MINAS GERAIS (DEOP-MG)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'DEPARTAMENTO EDIFICAÇÕES ESTRADAS RODAGEM ESTADO MINAS GERAIS (DER-MG)'],
        'DEPARTAMENTO DE ESTRADAS DE RODAGEM DO ESTADO DE MINAS GERAIS (DER)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'EDUCAÇÃO, ASSISTÊNCIA SOCIAL'],
        'EDUCAÇÃO'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'EMPRESA ASSISTÊNCIA TÉCNICA EXTENSÃO RURAL ESTADO MINAS GERAIS (EMATER-MG)'],
        'EMPRESA DE ASSISTÊNCIA TÉCNICA E EXTENSÃO RURAL DO ESTADO DE MINAS GERAIS (EMATER-MG)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'ENERGIA',
        'ENERGIA ELÉTRICA'],
        'ENERGIA E ENERGIA ELÉTRICA'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'FUNDAÇÃO ESTADUAL DE EDUCAÇÃO RURAL HELENA ANTIPOFF'],
        'FUNDAÇÃO HELENA ANTIPOFF (FHA)'
    )
    
    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'FUNDAÇÃO JOÃO PINHEIRO (FJP))'],
        'FUNDAÇÃO JOÃO PINHEIRO (FJP)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'FUNDAÇÃO RURAL MINEIRA COLONIZAÇÃO E DESENVOLVIMENTO AGRÁRIO (RURALMINAS)'],
        'FUNDAÇÃO RURAL MINEIRA (RURALMINAS)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'GOVERNADOR',
        'GOVERNADORIA DO ESTADO'],
        'GOVERNADOR E GOVERNADORIA DO ESTADO'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'IMOVEL'],
        'IMÓVEL'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'INSTITUTO DE DESENVOLVIMENTO DO NORTE E NORDESTE DE MINAS GERAIS'],
        'INSTITUTO DE DESENVOLVIMENTO DO NORTE E NORDESTE DE MINAS GERAIS (IDENE)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'INSTITUTO DE PESOS E MEDIDAS DO ESTADO DE MINAS GERAIS (IPEM-MG)'],
        'INSTITUTO DE METROLOGIA E QUALIDADE DO ESTADO DE MINAS GERAIS (IPEM)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES MILITARES DE MINAS GERAIS (IPSM)'],
        'INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES MILITARES DO ESTADO DE MINAS GERAIS (IPSM)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'INSTITUTO MINEIRO DE GESTÃO DAS ÁGUAS'],
        'INSTITUTO MINEIRO DE GESTÃO DAS ÁGUAS (IGAM)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'INSTITUTO ESTADUAL DE DESENVOLVIMENTO DE RECURSOS HUMANOS(IEDRHU)'],
        'INSTITUTO ESTADUAL DE DESENVOLVIMENTO DE RECURSOS HUMANOS (IEDRHU)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'INSTITUTO DE DESENVOLVIMENTO INTEGRADO DE MINAS GERAIS(INDI)'],
        'INSTITUTO DE DESENVOLVIMENTO INTEGRADO DE MINAS GERAIS (INDI)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'MINISTÉRIO PÚBLICO DO ESTADO DE MINAS GERAIS (MPMG) TRIBUNAL DE CONTAS DO ESTADO DE MINAS GERAIS (TCEMG)'],
        'MINISTÉRIO PÚBLICO DO ESTADO DE MINAS GERAIS (MPMG)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'MUNICÍPIOS E  DESENVOLVIMENTO REGIONAL'],
        'MUNICÍPIOS E DESENVOLVIMENTO REGIONAL'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'PATRIÔNIO'],
        'PATRIMÔNIO'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'PESSOAL EDUCAÇÃO'],
        'PESSOAL'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'POLÍCIA CIVIL'],
        'POLÍCIA CIVIL DO ESTADO DE MINAS GERAIS (PCMG)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'PROCURADORIA FISCAL DO ESTADO'],
        'PROCURADORIA-GERAL DA FAZENDA ESTADUAL'
     )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'PROCURADORIA-GERAL DO ESTADO'],
        'PROCURADORIA-GERAL DO ESTADO (PGE)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'SECRETARIA DE ESTADO DE AGRICULTURA E PECUÁRIA',
        'SECRETARIA DE ESTADO DE AGRICULTURA PECUÁRIA E ABASTECIMENTO (SEAPA)'],
        'SECRETARIA DE ESTADO DE AGRICULTURA E PECUÁRIA (SEAPA)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'SECRETARIA DE ESTADO DA CASA CIVIL E COMUNICAÇÃO SOCIAL'],
        'SECRETARIA DE ESTADO DA CASA CIVIL'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'SECRETARIA DA EDUCAÇÃO'],
        'SECRETARIA DE ESTADO DE EDUCAÇÃO (SEE)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'SECRETARIA DE FINANÇAS'],
        'SECRETARIA DE ESTADO DE FAZENDA (SEF)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'SECRETARIA DE SAÚDE E ASSISTÊNCIA'],
        'SECRETARIA DE ESTADO DE SAÚDE (SES)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'SECRETARIA DE ESTADO DE SEGURANÇA PÚBLICA (SESP)',
        'SECRETARIA DE SEGURANÇA PÚBLICA'],
        'SECRETARIA DE ESTADO DE SEGURANÇA PÚBLICA (SSPMG)'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'SEGURANÇA PÚBLICA PESSOA COM DEFICIÊNCIA'],
        'SEGURANÇA PÚBLICA'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'TRANSPORTE'],
        'TRANSPORTE E TRÂNSITO'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'TRIBUTO'],
        'TRIBUTOS'
    )

    norma['AssuntoGeral'] = substitui_em_set(norma['AssuntoGeral'], [
        'VICE-GOVERNADOR',
        'VICE-GOVERNADORIA'],
        'VICE-GOVERNADOR E VICE-GOVERNADORIA'
    )

### Remove normas sem assunto ou que falam de utilidade pública/próprio público

In [15]:
# Remove normas sem assuntos:
normasComAssuntos = [norma for norma in normas if len(norma['AssuntoGeral']) > 0]

# Remove normas que falam de utilidade pública/próprio público:
normasSemUtilidadeProprioPublico = [norma for norma in normasComAssuntos if set(['UTILIDADE PÚBLICA', 'PRÓPRIO PÚBLICO']).isdisjoint(norma['AssuntoGeral'])]

In [16]:
print_stats_multiplo(normasSemUtilidadeProprioPublico, top=20)

Quantidade de normas 10147
Quantidade de normas com AssuntoGeral: 10147
Quantidade de AssuntoGeral: 283
Quantidade de AssuntoGeral em mais de 5 normas: 174
Quantidade de normas com mais de 1 AssuntoGeral: 3750
AssuntoGeral comuns:
[ ('IMÓVEL', 2038),
  ('PESSOAL', 1731),
  ('CRÉDITO', 1486),
  ('TRIBUTOS', 578),
  ('EXECUTIVO', 549),
  ('SECRETARIA DE ESTADO DE EDUCAÇÃO (SEE)', 543),
  ('ORGANIZAÇÃO ADMINISTRATIVA', 541),
  ('ESTABELECIMENTO DE ENSINO', 535),
  ('ADMINISTRAÇÃO ESTADUAL', 355),
  ('DIVISÃO ADMINISTRATIVA', 346),
  ('SAÚDE PÚBLICA', 322),
  ('HOMENAGEM', 241),
  ('MEIO AMBIENTE', 221),
  ('JUDICIÁRIO', 212),
  ('ORÇAMENTO', 204),
  ('TRANSPORTE E TRÂNSITO', 199),
  ('CALENDÁRIO', 193),
  ('AUXÍLIO FINANCEIRO', 190),
  ('FINANÇAS PÚBLICAS', 189),
  ('POLÍCIA MILITAR DO ESTADO DE MINAS GERAIS (PMMG)', 178)]

Pares de AssuntoGeral comuns:
[ (('PESSOAL', 'SECRETARIA DE ESTADO DE EDUCAÇÃO (SEE)'), 522),
  (('ESTABELECIMENTO DE ENSINO', 'PESSOAL'), 412),
  ( ('ESTABELECIMENTO 

In [17]:
# Assuntos que aparecem 20 ou mais vezes:
minC = 20
assuntosComuns = [a for a in sorted(Counter([assunto for norma in normasSemUtilidadeProprioPublico for assunto in norma['AssuntoGeral']]).items()) if a[1] >= minC]
display(len(assuntosComuns))
display(assuntosComuns)

80

[('ADMINISTRAÇÃO ESTADUAL', 355),
 ('ADMINISTRAÇÃO INDIRETA', 122),
 ('ADMINISTRAÇÃO MUNICIPAL', 60),
 ('ADVOCACIA-GERAL DO ESTADO DE MINAS GERAIS (AGE)', 21),
 ('AGROPECUÁRIA', 157),
 ('ALIMENTAÇÃO', 28),
 ('ASSEMBLEIA LEGISLATIVA DO ESTADO DE MINAS GERAIS (ALMG)', 139),
 ('ASSISTÊNCIA SOCIAL', 91),
 ('AUXÍLIO FINANCEIRO', 190),
 ('BARRAGEM HÍDRICA', 29),
 ('BENEFÍCIO PESSOAL', 156),
 ('CALAMIDADE PÚBLICA', 50),
 ('CALENDÁRIO', 193),
 ('CARTÓRIO', 50),
 ('COMUNICAÇÃO', 22),
 ('CONSELHO ESTADUAL', 84),
 ('CONSTITUIÇÃO ESTADUAL', 173),
 ('CONTRATO', 38),
 ('CONVÊNIO', 28),
 ('CORPO DE BOMBEIROS MILITAR DO ESTADO DE MINAS GERAIS (CBMMG)', 61),
 ('CRIANÇA E ADOLESCENTE', 74),
 ('CRÉDITO', 1486),
 ('CULTURA', 122),
 ('DEFENSORIA PÚBLICA DO ESTADO DE MINAS GERAIS (DPMG)', 25),
 ('DEFESA DO CONSUMIDOR', 133),
 ('DIREITOS HUMANOS', 154),
 ('DIVISÃO ADMINISTRATIVA', 346),
 ('EDUCAÇÃO', 160),
 ('ENSINO SUPERIOR', 98),
 ('ESPORTE E LAZER', 41),
 ('ESTABELECIMENTO DE ENSINO', 535),
 ('EXECUTIVO',

In [18]:
def filtra_em_set(lista, tokens_permitidos):
    return set([token for token in lista if token in tokens_permitidos])

In [19]:
for norma in normasSemUtilidadeProprioPublico:
    norma['AssuntoGeral'] = list(filtra_em_set(norma['AssuntoGeral'], [x[0] for x in assuntosComuns]))
    norma['Tema'] = list(norma['Tema'])
    norma['Indexacao'] = list(norma['Indexacao'])

In [20]:
normasSemUtilidadeProprioPublicoReduzida = [norma for norma in normasSemUtilidadeProprioPublico if len(norma['AssuntoGeral']) > 0]

In [21]:
print_stats_multiplo(normasSemUtilidadeProprioPublicoReduzida, top=20)

Quantidade de normas 9963
Quantidade de normas com AssuntoGeral: 9963
Quantidade de AssuntoGeral: 80
Quantidade de AssuntoGeral em mais de 5 normas: 80
Quantidade de normas com mais de 1 AssuntoGeral: 3379
AssuntoGeral comuns:
[ ('IMÓVEL', 2038),
  ('PESSOAL', 1731),
  ('CRÉDITO', 1486),
  ('TRIBUTOS', 578),
  ('EXECUTIVO', 549),
  ('SECRETARIA DE ESTADO DE EDUCAÇÃO (SEE)', 543),
  ('ORGANIZAÇÃO ADMINISTRATIVA', 541),
  ('ESTABELECIMENTO DE ENSINO', 535),
  ('ADMINISTRAÇÃO ESTADUAL', 355),
  ('DIVISÃO ADMINISTRATIVA', 346),
  ('SAÚDE PÚBLICA', 322),
  ('HOMENAGEM', 241),
  ('MEIO AMBIENTE', 221),
  ('JUDICIÁRIO', 212),
  ('ORÇAMENTO', 204),
  ('TRANSPORTE E TRÂNSITO', 199),
  ('CALENDÁRIO', 193),
  ('AUXÍLIO FINANCEIRO', 190),
  ('FINANÇAS PÚBLICAS', 189),
  ('POLÍCIA MILITAR DO ESTADO DE MINAS GERAIS (PMMG)', 178)]

Pares de AssuntoGeral comuns:
[ (('PESSOAL', 'SECRETARIA DE ESTADO DE EDUCAÇÃO (SEE)'), 522),
  (('ESTABELECIMENTO DE ENSINO', 'PESSOAL'), 412),
  ( ('ESTABELECIMENTO DE E

In [24]:
dump_json("normas_80assuntos.json", normasSemUtilidadeProprioPublicoReduzida);